#### Imports

In [1]:
# Telegram Bot
import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

# Config Driver
from config.config_driver import ConfigDriver

# Amazon Driver
from amazon.amazon import Amazon

# Definitions
from definitions import *

#### Variables Main

In [ ]:
API_TOKEN = 'Your_Token_Here'
bot = telebot.TeleBot(API_TOKEN)

config_driver = ConfigDriver()
driver = config_driver.get_driver()

amazon_config = Amazon(driver)
offers_day = False
search_product = False

#### Main

In [3]:
def prepare_markup():
    """ Method to prepare the markup """
    
    markup = InlineKeyboardMarkup()
    
    option1 = InlineKeyboardButton("Pesquisar Preço de Produto", callback_data="opt1")
    option2 = InlineKeyboardButton("Ofertas do Dia", callback_data="opt2")

    markup.add(option1, option2)
        
    return markup

def get_function_details(message):
    """ Method to get the function details """
    
    global offers_day, search_product
    
    response = {}
    
    if message.text == '/start':
        response['message'] = "Assistente Virtual da Amazon inicializado!"
    
    elif message.text in SAUDACOES:
        user = message.from_user.first_name
        
        markup = InlineKeyboardMarkup()
        
        response['markup'] = prepare_markup()
        
        response['text'] = f"Eae {user}! Em que posso te ajudar?"
    elif offers_day and amazon_config.is_departament(message.text):
        departament_choice = message.text
        response['products'] = amazon_config.get_products_of_departament(departament_choice)
        offers_day = False
    elif search_product:
        search_product = False
        product_name = message.text
        product = amazon_config.get_top_products(product_name)
        if len(product) == 0:
            response['message'] = "Desculpe, não foi possível encontrar o produto que você deseja."
        else:
            response['search_product'] = product
    else:
        response['message'] = "Desculpe, eu não entendi o que você disse."
        
    return response

################################################################################
@bot.message_handler(func=lambda message: True)
def reply_message(message):
    """ Method to reply the message """
    
    response = get_function_details(message)
    
    if 'markup' in response:
        bot.reply_to(message, response['text'], reply_markup=response["markup"])
    elif 'message' in response:
        bot.send_message(message.chat.id, response['message'])
        
    elif 'products' in response:
        list_products = []

        for product in response['products']:
            product_name, product_link, product_image, product_discount = product

            # Skip products with any missing information
            if not all([product_name, product_link, product_image, product_discount]):
                continue

            list_products.append((product_name, product_link, product_image, product_discount))

        bot.send_message(message.chat.id, f"🛒 *Foi encontrado* {len(list_products)} *produtos!*", parse_mode='Markdown')
        bot.send_message(message.chat.id, "📢 *Aqui estão as ofertas do dia!*", parse_mode='Markdown')

        for product in list_products:
            product_name, product_link, product_image, product_discount = product

            message_text = (
                f"📦 *Produto:* {product_name}\n"
                f"💸 *Desconto:* {product_discount}\n"
                f"🔗 [Clique aqui]({product_link})"
            )
            
            try:
                bot.send_photo(
                    chat_id=message.chat.id,
                    photo=product_image,
                    caption=message_text,
                    parse_mode='Markdown'
                )
            except Exception as e:
                error_message = f"⚠️ Erro ao enviar o produto: {product_name}"
                bot.send_message(chat_id=message.chat.id, text=error_message, parse_mode='Markdown')
                print(f"Erro ao enviar o produto '{product_name}': {e}")

        bot.send_message(message.chat.id, "🤖 *O que posso te ajudar?*", reply_markup=prepare_markup(), parse_mode='Markdown')

    elif 'search_product' in response:
        products = response['search_product']
        
        for product in products:
            product_name = product.get('name', 'No Name')
            product_link = product.get('link', 'No Link')
            product_image = product.get('image', '')
            product_price = product.get('price', 'Preço não disponível')

            message_text = (
                f"📦 *Produto:* {product_name}\n"
                f"💰 *Preço:* R${product_price}\n"
                f"🔗 [Clique aqui]({product_link})"
            )
            
            try:
                bot.send_photo(
                    chat_id=message.chat.id,
                    photo=product_image,
                    caption=message_text,
                    parse_mode='Markdown'
                )
            except Exception as e:
                bot.send_message(chat_id=message.chat.id, text=f"Erro ao enviar o produto: {product_name}")
                print(f"Erro ao enviar o produto: {e}")
        
        bot.send_message(message.chat.id, "🤖 *O que posso te ajudar?*", reply_markup=prepare_markup(), parse_mode='Markdown')

################################################################################
@bot.callback_query_handler(func=lambda call: True)
def handle_callback_query(call):
    """ Method to handle the callback query """
    
    global offers_day, search_product
    
    if call.data == "opt1":
        bot.answer_callback_query(call.id, "Pesquisar Preço de Produto!")
        search_product = True

        bot.send_message(call.message.chat.id, "Digite o nome do produto que você deseja pesquisar!")
    
    elif call.data == "opt2":
        
        offers_day = True
        
        bot.answer_callback_query(call.id, "Ofertas do Dia!")        
        departments = amazon_config.get_departaments_offer_of_the_day()
        
        departments_formatted = "\n".join(departments)
        
        bot.send_message(call.message.chat.id, f"Departamentos disponíveis:\n{departments_formatted}")
        bot.send_message(call.message.chat.id, "Qual departamento você deseja ver as ofertas?")

################################################################################

#### Run Bot

In [ ]:
bot.polling()